# Panel Regression 
## Regression with data that is both cross-sectional and time-series

In [99]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

from linearmodels.panel import PanelOLS
from linearmodels.panel import RandomEffects
from linearmodels.panel import compare

df = pd.read_csv('/Users/robertwrobel/Code/Python4Statistics/Notebooks/panel_data.csv')
df.head(15)

,Company,Year,Marketing,RD,Revenue
0,Company A,2010,105.01,160.73,498.51
1,Company A,2011,106.93,166.82,536.58
2,Company A,2012,86.40,182.49,552.30
3,Company A,2013,104.65,181.61,566.84
4,Company A,2014,105.86,108.67,382.67
5,Company A,2015,85.71,121.87,391.84
6,Company A,2016,137.32,165.45,582.64
7,Company A,2017,109.48,165.41,521.05
8,Company A,2018,76.17,165.45,482.68
9,Company A,2019,113.13,265.58,761.72


In [101]:
df = df.set_index(['Company', 'Year'])
df.head(15)

Marketing      RD  Revenue
Company   Year                            
Company A 2010     105.01  160.73   498.51
          2011     106.93  166.82   536.58
          2012      86.40  182.49   552.30
          2013     104.65  181.61   566.84
          2014     105.86  108.67   382.67
          2015      85.71  121.87   391.84
          2016     137.32  165.45   582.64
          2017     109.48  165.41   521.05
          2018      76.17  165.45   482.68
          2019     113.13  265.58   761.72
Company B 2010      80.51  167.13   479.18
          2011     115.74  184.07   577.97
          2012     123.17  178.62   545.31
          2013      83.59  169.54   488.13
          2014     119.27  140.54   485.57

In [103]:
X = df[['Marketing', 'RD']]
X = sm.add_constant(X)
y = df['Revenue']

In [105]:
#OLS model for baseline
model = sm.OLS(y,X)
ols_results = model.fit()
ols_results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                Revenue   R-squared:                       0.984
Model:                            OLS   Adj. R-squared:                  0.984
Method:                 Least Squares   F-statistic:                     1465.
Date:                Wed, 29 May 2024   Prob (F-statistic):           4.54e-43
Time:                        13:18:53   Log-Likelihood:                -185.18
No. Observations:                  50   AIC:                             376.4
Df Residuals:                      47   BIC:                             382.1
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.7485     10.935      0.068      0.946     -21.249      22.746
Marketing      1.3436      0.081     16.587      0.000       1.181       1.507
RD             2.2967      0.044     51.961      0.000       2.208       2.386
==============================================================================
Omnibus:                        4.372   Durbin-Watson:                   2.244
Prob(Omnibus):                  0.112   Jarque-Bera (JB):                3.270
Skew:                          -0.495   Prob(JB):                        0.195
Kurtosis:                       3.767   Cond. No.                     1.43e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.43e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [107]:
#Fixed effects - Differences between entities
#F test for poolability: H0 = individual effects are minimal / fixed effects not needed, HA = fixed effects needed, if p-value < alpha, fixed effects are appropriate
model = PanelOLS(y, X, entity_effects=True)  
fe_results1 = model.fit()
fe_results1.summary

Dep. Variable:,Revenue,R-squared:,0.9848
Estimator:,PanelOLS,R-squared (Between):,0.9744
No. Observations:,50,R-squared (Within):,0.9848
Date:,"Wed, May 29 2024",R-squared (Overall):,0.9842
Time:,13:18:53,Log-likelihood,-182.55
Cov. Estimator:,Unadjusted,,
,,F-statistic:,1396.8
Entities:,5,P-value,0.0000
Avg Obs:,10.0000,Distribution:,"F(2,43)"
Min Obs:,10.0000,,
Max Obs:,10.0000,F-statistic (robust):,1396.8


In [109]:
#Fixed effects -  Differences between time periods
model = PanelOLS(y, X, time_effects=True)  
fe_results2 = model.fit()
fe_results2.summary

Dep. Variable:,Revenue,R-squared:,0.9860
Estimator:,PanelOLS,R-squared (Between):,0.9756
No. Observations:,50,R-squared (Within):,0.9844
Date:,"Wed, May 29 2024",R-squared (Overall):,0.9839
Time:,13:18:53,Log-likelihood,-179.10
Cov. Estimator:,Unadjusted,,
,,F-statistic:,1334.7
Entities:,5,P-value,0.0000
Avg Obs:,10.0000,Distribution:,"F(2,38)"
Min Obs:,10.0000,,
Max Obs:,10.0000,F-statistic (robust):,1334.7


In [111]:
#Fixed effects - Differences between entities and time periods
model = PanelOLS(y, X, entity_effects=True, time_effects=True)  
fe_results3 = model.fit()
fe_results3.summary

Dep. Variable:,Revenue,R-squared:,0.9868
Estimator:,PanelOLS,R-squared (Between):,0.9745
No. Observations:,50,R-squared (Within):,0.9845
Date:,"Wed, May 29 2024",R-squared (Overall):,0.9838
Time:,13:18:53,Log-likelihood,-175.71
Cov. Estimator:,Unadjusted,,
,,F-statistic:,1271.0
Entities:,5,P-value,0.0000
Avg Obs:,10.0000,Distribution:,"F(2,34)"
Min Obs:,10.0000,,
Max Obs:,10.0000,F-statistic (robust):,1271.0


In [113]:
#Random effects 
model = RandomEffects(y, X)
re_results = model.fit()
re_results.summary

Dep. Variable:,Revenue,R-squared:,0.9845
Estimator:,RandomEffects,R-squared (Between):,0.9750
No. Observations:,50,R-squared (Within):,0.9848
Date:,"Wed, May 29 2024",R-squared (Overall):,0.9842
Time:,13:18:53,Log-likelihood,-183.99
Cov. Estimator:,Unadjusted,,
,,F-statistic:,1491.9
Entities:,5,P-value,0.0000
Avg Obs:,10.0000,Distribution:,"F(2,47)"
Min Obs:,10.0000,,
Max Obs:,10.0000,F-statistic (robust):,1491.9


In [115]:
comparison = compare({'Fixed Effects': fe_results1, 'Random Effects': re_results})
comparison

,Fixed Effects,Random Effects
Dep. Variable,Revenue,Revenue
Estimator,PanelOLS,RandomEffects
No. Observations,50,50
Cov. Est.,Unadjusted,Unadjusted
R-squared,0.9848,0.9845
R-Squared (Within),0.9848,0.9848
R-Squared (Between),0.9744,0.9750
R-Squared (Overall),0.9842,0.9842
F-statistic,1396.8,1491.9
P-value (F-stat),0.0000,0.0000
